# Day 3: Risk Parity Portfolio Optimization

## Learning Objectives
- Understand the concept of risk contribution and marginal risk
- Implement Equal Risk Contribution (ERC) portfolios
- Build risk parity optimization from scratch
- Compare risk parity to mean-variance optimization
- Apply risk parity to multi-asset portfolios

## Topics Covered
1. Risk Contribution Fundamentals
2. Equal Risk Contribution (ERC)
3. Risk Parity Optimization Methods
4. Practical Implementation
5. Extensions and Variations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

---
## Part 1: Risk Contribution Fundamentals

### Understanding Portfolio Risk Decomposition

Portfolio volatility can be decomposed into contributions from each asset:

**Portfolio Variance:**
$$\sigma_p^2 = w^T \Sigma w$$

**Marginal Risk Contribution (MRC):**
$$MRC_i = \frac{\partial \sigma_p}{\partial w_i} = \frac{(\Sigma w)_i}{\sigma_p}$$

**Total Risk Contribution (TRC):**
$$TRC_i = w_i \times MRC_i = \frac{w_i (\Sigma w)_i}{\sigma_p}$$

**Key Property - Euler Decomposition:**
$$\sum_{i=1}^{n} TRC_i = \sigma_p$$

**Percentage Risk Contribution:**
$$RC_i = \frac{TRC_i}{\sigma_p} = \frac{w_i (\Sigma w)_i}{w^T \Sigma w}$$

In [ ]:
class RiskContributionAnalyzer:
    """
    Analyze risk contributions of portfolio assets.
    """
    
    def __init__(self, returns: pd.DataFrame):
        """
        Initialize with return series.
        
        Parameters:
        -----------
        returns : pd.DataFrame
            Asset returns (T x N)
        """
        self.returns = returns
        self.cov_matrix = returns.cov() * 252  # Annualized
        self.asset_names = returns.columns.tolist()
        self.n_assets = len(self.asset_names)
    
    def portfolio_volatility(self, weights: np.ndarray) -> float:
        """Calculate portfolio volatility."""
        return np.sqrt(weights @ self.cov_matrix.values @ weights)
    
    def marginal_risk_contribution(self, weights: np.ndarray) -> np.ndarray:
        """
        Calculate Marginal Risk Contribution (MRC) for each asset.
        MRC_i = d(sigma_p) / d(w_i)
        """
        sigma_p = self.portfolio_volatility(weights)
        mrc = (self.cov_matrix.values @ weights) / sigma_p
        return mrc
    
    def total_risk_contribution(self, weights: np.ndarray) -> np.ndarray:
        """
        Calculate Total Risk Contribution (TRC) for each asset.
        TRC_i = w_i * MRC_i
        """
        mrc = self.marginal_risk_contribution(weights)
        trc = weights * mrc
        return trc
    
    def percentage_risk_contribution(self, weights: np.ndarray) -> np.ndarray:
        """
        Calculate Percentage Risk Contribution (RC%) for each asset.
        RC_i = TRC_i / sigma_p
        """
        trc = self.total_risk_contribution(weights)
        sigma_p = self.portfolio_volatility(weights)
        return trc / sigma_p
    
    def risk_decomposition_report(self, weights: np.ndarray) -> pd.DataFrame:
        """
        Generate comprehensive risk decomposition report.
        """
        sigma_p = self.portfolio_volatility(weights)
        mrc = self.marginal_risk_contribution(weights)
        trc = self.total_risk_contribution(weights)
        prc = self.percentage_risk_contribution(weights)
        
        report = pd.DataFrame({
            'Asset': self.asset_names,
            'Weight (%)': weights * 100,
            'Volatility (%)': np.sqrt(np.diag(self.cov_matrix.values)) * 100,
            'MRC': mrc,
            'TRC': trc,
            'Risk Contribution (%)': prc * 100
        }).set_index('Asset')
        
        # Add totals
        report.loc['TOTAL'] = [
            report['Weight (%)'].sum(),
            np.nan,
            np.nan,
            report['TRC'].sum(),
            report['Risk Contribution (%)'].sum()
        ]
        
        print(f"Portfolio Volatility: {sigma_p*100:.2f}%")
        print(f"Sum of TRC (should equal sigma_p): {trc.sum()*100:.2f}%")
        
        return report

In [ ]:
# Download sample data
tickers = ['SPY', 'TLT', 'GLD', 'VNQ', 'EFA']  # Stocks, Bonds, Gold, REITs, International
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

print("Downloading price data...")
data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
returns = data.pct_change().dropna()

print(f"Data period: {returns.index[0].strftime('%Y-%m-%d')} to {returns.index[-1].strftime('%Y-%m-%d')}")
print(f"Number of observations: {len(returns)}")
returns.head()

In [ ]:
# Initialize analyzer
analyzer = RiskContributionAnalyzer(returns)

# Compare risk contributions for different weight schemes
n_assets = len(tickers)

# Equal Weight Portfolio
equal_weights = np.ones(n_assets) / n_assets
print("="*60)
print("EQUAL WEIGHT PORTFOLIO")
print("="*60)
ew_report = analyzer.risk_decomposition_report(equal_weights)
print(ew_report.round(2))

In [ ]:
# 60/40 Portfolio (assuming SPY is first, TLT is second)
portfolio_60_40 = np.array([0.60, 0.40, 0.0, 0.0, 0.0])
print("\n" + "="*60)
print("60/40 PORTFOLIO (SPY/TLT)")
print("="*60)
p60_40_report = analyzer.risk_decomposition_report(portfolio_60_40)
print(p60_40_report.round(2))

In [ ]:
# Visualize risk contributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Equal Weight
ax1 = axes[0]
ew_rc = analyzer.percentage_risk_contribution(equal_weights)
colors = plt.cm.Set2(np.linspace(0, 1, n_assets))
ax1.bar(tickers, equal_weights * 100, alpha=0.7, label='Weight', color=colors)
ax1.bar(tickers, ew_rc * 100, alpha=0.5, label='Risk Contribution', 
        edgecolor='black', linewidth=2, fill=False)
ax1.set_ylabel('Percentage (%)')
ax1.set_title('Equal Weight: Weights vs Risk Contributions')
ax1.legend()
ax1.axhline(y=20, color='red', linestyle='--', alpha=0.5, label='Equal (20%)')

# 60/40
ax2 = axes[1]
p60_40_rc = analyzer.percentage_risk_contribution(portfolio_60_40)
width = 0.35
x = np.arange(n_assets)
ax2.bar(x - width/2, portfolio_60_40 * 100, width, alpha=0.7, label='Weight', color=colors)
ax2.bar(x + width/2, p60_40_rc * 100, width, alpha=0.7, label='Risk Contribution', color='red')
ax2.set_xticks(x)
ax2.set_xticklabels(tickers)
ax2.set_ylabel('Percentage (%)')
ax2.set_title('60/40 Portfolio: Weights vs Risk Contributions')
ax2.legend()

plt.tight_layout()
plt.show()

print("\nKey Insight: Equal weights ≠ Equal risk contributions!")
print(f"SPY contributes {ew_rc[0]*100:.1f}% of risk with only {equal_weights[0]*100:.1f}% weight")

---
## Part 2: Equal Risk Contribution (ERC) Portfolio

### The Risk Parity Concept

**Objective:** Find weights such that each asset contributes equally to portfolio risk.

**Mathematical Formulation:**

For an ERC portfolio with $n$ assets:
$$RC_i = \frac{1}{n} \quad \forall i$$

This means:
$$w_i (\Sigma w)_i = w_j (\Sigma w)_j \quad \forall i,j$$

### Optimization Problem

**Minimize the sum of squared differences in risk contributions:**
$$\min_w \sum_{i=1}^{n} \sum_{j=1}^{n} (w_i (\Sigma w)_i - w_j (\Sigma w)_j)^2$$

Subject to:
- $\sum_{i=1}^{n} w_i = 1$ (fully invested)
- $w_i \geq 0$ (long only)

In [ ]:
class RiskParityOptimizer:
    """
    Risk Parity / Equal Risk Contribution Portfolio Optimizer.
    """
    
    def __init__(self, cov_matrix: np.ndarray, asset_names: list = None):
        """
        Initialize optimizer with covariance matrix.
        
        Parameters:
        -----------
        cov_matrix : np.ndarray
            Covariance matrix (N x N)
        asset_names : list
            Names of assets
        """
        self.cov_matrix = np.array(cov_matrix)
        self.n_assets = len(cov_matrix)
        self.asset_names = asset_names or [f'Asset_{i}' for i in range(self.n_assets)]
    
    def portfolio_volatility(self, weights: np.ndarray) -> float:
        """Calculate portfolio volatility."""
        return np.sqrt(weights @ self.cov_matrix @ weights)
    
    def risk_contribution(self, weights: np.ndarray) -> np.ndarray:
        """
        Calculate risk contribution for each asset.
        """
        sigma_p = self.portfolio_volatility(weights)
        # Marginal contribution to risk
        mrc = (self.cov_matrix @ weights) / sigma_p
        # Risk contribution
        rc = weights * mrc
        return rc / sigma_p  # Percentage contribution
    
    def _erc_objective(self, weights: np.ndarray) -> float:
        """
        Objective function for ERC optimization.
        Minimize sum of squared differences in risk contributions.
        """
        rc = self.risk_contribution(weights)
        target_rc = 1.0 / self.n_assets
        return np.sum((rc - target_rc) ** 2)
    
    def _erc_objective_v2(self, weights: np.ndarray) -> float:
        """
        Alternative objective: Minimize pairwise differences.
        """
        sigma_sq = weights @ self.cov_matrix @ weights
        risk_contrib = weights * (self.cov_matrix @ weights)
        
        obj = 0
        for i in range(self.n_assets):
            for j in range(i+1, self.n_assets):
                obj += (risk_contrib[i] - risk_contrib[j]) ** 2
        return obj
    
    def optimize_erc(self, method: str = 'SLSQP') -> dict:
        """
        Optimize for Equal Risk Contribution portfolio.
        
        Parameters:
        -----------
        method : str
            Optimization method ('SLSQP', 'trust-constr')
        
        Returns:
        --------
        dict : Optimization results
        """
        # Initial guess (inverse volatility weighted)
        vols = np.sqrt(np.diag(self.cov_matrix))
        x0 = (1/vols) / np.sum(1/vols)
        
        # Constraints
        constraints = [
            {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}  # Fully invested
        ]
        
        # Bounds (long only)
        bounds = [(0.001, 1.0) for _ in range(self.n_assets)]
        
        # Optimize
        result = minimize(
            self._erc_objective,
            x0,
            method=method,
            bounds=bounds,
            constraints=constraints,
            options={'maxiter': 1000, 'ftol': 1e-12}
        )
        
        optimal_weights = result.x / result.x.sum()  # Normalize
        
        return {
            'weights': optimal_weights,
            'volatility': self.portfolio_volatility(optimal_weights),
            'risk_contributions': self.risk_contribution(optimal_weights),
            'success': result.success,
            'message': result.message
        }

In [ ]:
# Optimize for Risk Parity
cov_matrix = returns.cov().values * 252  # Annualized
rp_optimizer = RiskParityOptimizer(cov_matrix, tickers)

# Find ERC weights
erc_result = rp_optimizer.optimize_erc()

print("EQUAL RISK CONTRIBUTION (ERC) PORTFOLIO")
print("="*50)
print(f"\nOptimization Success: {erc_result['success']}")
print(f"Portfolio Volatility: {erc_result['volatility']*100:.2f}%\n")

erc_df = pd.DataFrame({
    'Asset': tickers,
    'Weight (%)': erc_result['weights'] * 100,
    'Risk Contribution (%)': erc_result['risk_contributions'] * 100,
    'Target RC (%)': np.ones(len(tickers)) * 100 / len(tickers)
}).set_index('Asset')

print(erc_df.round(2))
print(f"\nSum of weights: {erc_result['weights'].sum()*100:.2f}%")
print(f"Sum of risk contributions: {erc_result['risk_contributions'].sum()*100:.2f}%")

In [ ]:
# Visualize ERC vs Equal Weight
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Weight comparison
ax1 = axes[0]
x = np.arange(len(tickers))
width = 0.35
ax1.bar(x - width/2, equal_weights * 100, width, label='Equal Weight', alpha=0.8)
ax1.bar(x + width/2, erc_result['weights'] * 100, width, label='Risk Parity', alpha=0.8)
ax1.set_xticks(x)
ax1.set_xticklabels(tickers)
ax1.set_ylabel('Weight (%)')
ax1.set_title('Portfolio Weights')
ax1.legend()

# Risk contribution comparison
ax2 = axes[1]
ew_rc = analyzer.percentage_risk_contribution(equal_weights)
ax2.bar(x - width/2, ew_rc * 100, width, label='Equal Weight', alpha=0.8)
ax2.bar(x + width/2, erc_result['risk_contributions'] * 100, width, label='Risk Parity', alpha=0.8)
ax2.axhline(y=20, color='red', linestyle='--', alpha=0.7, label='Target (20%)')
ax2.set_xticks(x)
ax2.set_xticklabels(tickers)
ax2.set_ylabel('Risk Contribution (%)')
ax2.set_title('Risk Contributions')
ax2.legend()

# Pie charts
ax3 = axes[2]
ax3.pie(erc_result['risk_contributions'], labels=tickers, autopct='%1.1f%%',
        colors=plt.cm.Set2(np.linspace(0, 1, len(tickers))))
ax3.set_title('Risk Parity: Risk Contributions')

plt.tight_layout()
plt.show()

---
## Part 3: Risk Parity Optimization Methods

### Method 1: Newton-Raphson Approach (Spinu 2013)

The risk budgeting problem can be reformulated as:
$$\min_y \frac{1}{2} y^T \Sigma y - \sum_{i=1}^{n} b_i \log(y_i)$$

where $b_i$ is the target risk budget (for ERC, $b_i = 1/n$).

### Method 2: Cyclical Coordinate Descent (CCD)

Iteratively update each weight while keeping others fixed.

### Method 3: Analytical Solution (Special Case)

For diagonal covariance matrix (uncorrelated assets):
$$w_i^{ERC} = \frac{1/\sigma_i}{\sum_{j=1}^{n} 1/\sigma_j}$$

In [ ]:
class AdvancedRiskParity:
    """
    Advanced Risk Parity methods with multiple optimization approaches.
    """
    
    def __init__(self, cov_matrix: np.ndarray):
        self.cov_matrix = np.array(cov_matrix)
        self.n_assets = len(cov_matrix)
    
    def inverse_volatility_weights(self) -> np.ndarray:
        """
        Simple inverse volatility weights.
        Exact ERC solution for uncorrelated assets.
        """
        vols = np.sqrt(np.diag(self.cov_matrix))
        weights = (1 / vols) / np.sum(1 / vols)
        return weights
    
    def newton_raphson_rb(self, budgets: np.ndarray = None, 
                          max_iter: int = 100, tol: float = 1e-10) -> np.ndarray:
        """
        Newton-Raphson method for risk budgeting (Spinu 2013).
        
        Parameters:
        -----------
        budgets : np.ndarray
            Target risk budgets (default: equal)
        max_iter : int
            Maximum iterations
        tol : float
            Convergence tolerance
        """
        if budgets is None:
            budgets = np.ones(self.n_assets) / self.n_assets
        
        # Initialize with inverse vol weights
        y = self.inverse_volatility_weights()
        
        for iteration in range(max_iter):
            # Gradient
            Sigma_y = self.cov_matrix @ y
            grad = Sigma_y - budgets / y
            
            # Hessian
            hess = self.cov_matrix + np.diag(budgets / (y ** 2))
            
            # Newton step
            delta = np.linalg.solve(hess, grad)
            
            # Line search with backtracking
            alpha = 1.0
            while np.any(y - alpha * delta <= 0):
                alpha *= 0.5
            
            y_new = y - alpha * delta
            
            # Check convergence
            if np.max(np.abs(y_new - y)) < tol:
                break
            
            y = y_new
        
        # Normalize to sum to 1
        weights = y / np.sum(y)
        return weights
    
    def cyclical_coordinate_descent(self, budgets: np.ndarray = None,
                                    max_iter: int = 500, tol: float = 1e-10) -> np.ndarray:
        """
        Cyclical Coordinate Descent for risk parity.
        
        Updates one weight at a time while keeping others fixed.
        """
        if budgets is None:
            budgets = np.ones(self.n_assets) / self.n_assets
        
        # Initialize
        w = self.inverse_volatility_weights()
        
        for iteration in range(max_iter):
            w_old = w.copy()
            
            for i in range(self.n_assets):
                # Compute portfolio variance contribution terms
                sigma_i_sq = self.cov_matrix[i, i]
                cov_i_rest = np.delete(self.cov_matrix[i, :], i)
                w_rest = np.delete(w, i)
                
                # Quadratic equation coefficients
                a = sigma_i_sq
                b = 2 * np.dot(cov_i_rest, w_rest)
                c_term = -budgets[i] * np.sqrt(w @ self.cov_matrix @ w)
                
                # Solve quadratic
                discriminant = b**2 - 4*a*c_term
                if discriminant >= 0:
                    w[i] = (-b + np.sqrt(discriminant)) / (2 * a)
                    w[i] = max(w[i], 1e-10)  # Ensure positive
            
            # Check convergence
            if np.max(np.abs(w - w_old)) < tol:
                break
        
        # Normalize
        return w / np.sum(w)
    
    def risk_budgeting(self, budgets: np.ndarray, method: str = 'SLSQP') -> np.ndarray:
        """
        General risk budgeting portfolio.
        
        Parameters:
        -----------
        budgets : np.ndarray
            Target risk budgets (should sum to 1)
        method : str
            Optimization method
        """
        def objective(w):
            sigma_sq = w @ self.cov_matrix @ w
            risk_contrib = w * (self.cov_matrix @ w) / np.sqrt(sigma_sq)
            pct_contrib = risk_contrib / np.sqrt(sigma_sq)
            return np.sum((pct_contrib - budgets) ** 2)
        
        # Initial guess
        x0 = self.inverse_volatility_weights()
        
        # Constraints and bounds
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0.001, 1.0) for _ in range(self.n_assets)]
        
        result = minimize(objective, x0, method=method,
                         bounds=bounds, constraints=constraints)
        
        return result.x / result.x.sum()

In [ ]:
# Compare different risk parity methods
advanced_rp = AdvancedRiskParity(cov_matrix)

# Method 1: Simple Inverse Volatility
inv_vol_weights = advanced_rp.inverse_volatility_weights()

# Method 2: Newton-Raphson
nr_weights = advanced_rp.newton_raphson_rb()

# Method 3: Cyclical Coordinate Descent
ccd_weights = advanced_rp.cyclical_coordinate_descent()

# Method 4: SLSQP (from before)
slsqp_weights = erc_result['weights']

# Compare results
comparison_df = pd.DataFrame({
    'Inverse Vol': inv_vol_weights * 100,
    'Newton-Raphson': nr_weights * 100,
    'CCD': ccd_weights * 100,
    'SLSQP': slsqp_weights * 100
}, index=tickers)

print("WEIGHT COMPARISON ACROSS METHODS (%)")
print("="*60)
print(comparison_df.round(2))
print(f"\nMax weight difference: {comparison_df.max().max() - comparison_df.min().min():.4f}%")

In [ ]:
# Verify risk contributions for each method
def get_risk_contributions(weights, cov):
    sigma_p = np.sqrt(weights @ cov @ weights)
    mrc = (cov @ weights) / sigma_p
    return (weights * mrc) / sigma_p * 100

rc_comparison = pd.DataFrame({
    'Inverse Vol': get_risk_contributions(inv_vol_weights, cov_matrix),
    'Newton-Raphson': get_risk_contributions(nr_weights, cov_matrix),
    'CCD': get_risk_contributions(ccd_weights, cov_matrix),
    'SLSQP': get_risk_contributions(slsqp_weights, cov_matrix)
}, index=tickers)

print("\nRISK CONTRIBUTION COMPARISON (%)")
print("="*60)
print(rc_comparison.round(4))
print(f"\nTarget: {100/len(tickers):.2f}% each")
print(f"Max deviation from target (SLSQP): {abs(rc_comparison['SLSQP'] - 20).max():.4f}%")

---
## Part 4: Practical Implementation - Backtesting Risk Parity

### Comparing Portfolio Strategies
1. Equal Weight (1/N)
2. Risk Parity (ERC)
3. Inverse Volatility
4. Minimum Variance
5. 60/40 Stock/Bond

In [ ]:
class PortfolioBacktester:
    """
    Backtest multiple portfolio strategies.
    """
    
    def __init__(self, returns: pd.DataFrame, lookback: int = 252):
        """
        Initialize backtester.
        
        Parameters:
        -----------
        returns : pd.DataFrame
            Daily returns
        lookback : int
            Lookback period for covariance estimation
        """
        self.returns = returns
        self.lookback = lookback
        self.n_assets = returns.shape[1]
    
    def equal_weight(self, cov: np.ndarray = None) -> np.ndarray:
        """Equal weight portfolio."""
        return np.ones(self.n_assets) / self.n_assets
    
    def inverse_volatility(self, cov: np.ndarray) -> np.ndarray:
        """Inverse volatility weighted portfolio."""
        vols = np.sqrt(np.diag(cov))
        return (1 / vols) / np.sum(1 / vols)
    
    def risk_parity(self, cov: np.ndarray) -> np.ndarray:
        """Risk parity (ERC) portfolio."""
        rp = AdvancedRiskParity(cov)
        return rp.newton_raphson_rb()
    
    def minimum_variance(self, cov: np.ndarray) -> np.ndarray:
        """Minimum variance portfolio."""
        n = len(cov)
        
        def objective(w):
            return w @ cov @ w
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = [(0, 1) for _ in range(n)]
        x0 = np.ones(n) / n
        
        result = minimize(objective, x0, method='SLSQP',
                         bounds=bounds, constraints=constraints)
        return result.x
    
    def static_60_40(self, cov: np.ndarray = None) -> np.ndarray:
        """60/40 portfolio (first 2 assets)."""
        weights = np.zeros(self.n_assets)
        weights[0] = 0.60  # SPY
        weights[1] = 0.40  # TLT
        return weights
    
    def run_backtest(self, rebalance_freq: int = 21) -> dict:
        """
        Run backtest for all strategies.
        
        Parameters:
        -----------
        rebalance_freq : int
            Rebalancing frequency in days
        
        Returns:
        --------
        dict : Portfolio values and weights over time
        """
        strategies = {
            'Equal Weight': self.equal_weight,
            'Inverse Vol': self.inverse_volatility,
            'Risk Parity': self.risk_parity,
            'Min Variance': self.minimum_variance,
            '60/40': self.static_60_40
        }
        
        # Initialize
        start_idx = self.lookback
        dates = self.returns.index[start_idx:]
        
        portfolio_values = {name: [1.0] for name in strategies}
        weights_history = {name: [] for name in strategies}
        
        current_weights = {name: None for name in strategies}
        
        for i, date in enumerate(dates):
            # Rebalance
            if i % rebalance_freq == 0:
                hist_returns = self.returns.iloc[start_idx+i-self.lookback:start_idx+i]
                cov = hist_returns.cov().values * 252
                
                for name, strategy_func in strategies.items():
                    try:
                        current_weights[name] = strategy_func(cov)
                    except:
                        current_weights[name] = np.ones(self.n_assets) / self.n_assets
            
            # Calculate returns
            daily_returns = self.returns.loc[date].values
            
            for name in strategies:
                if current_weights[name] is not None:
                    port_return = np.dot(current_weights[name], daily_returns)
                    portfolio_values[name].append(
                        portfolio_values[name][-1] * (1 + port_return)
                    )
                    weights_history[name].append(current_weights[name].copy())
        
        # Convert to DataFrames
        portfolio_df = pd.DataFrame(portfolio_values, index=[dates[0] - pd.Timedelta(days=1)] + list(dates))
        
        return {
            'portfolio_values': portfolio_df,
            'weights_history': weights_history
        }

In [ ]:
# Run backtest
print("Running backtest...")
backtester = PortfolioBacktester(returns, lookback=252)
backtest_results = backtester.run_backtest(rebalance_freq=21)  # Monthly rebalancing

portfolio_values = backtest_results['portfolio_values']
print(f"Backtest period: {portfolio_values.index[0].strftime('%Y-%m-%d')} to {portfolio_values.index[-1].strftime('%Y-%m-%d')}")

In [ ]:
# Calculate performance metrics
def calculate_metrics(values: pd.Series) -> dict:
    """Calculate portfolio performance metrics."""
    returns = values.pct_change().dropna()
    
    total_return = (values.iloc[-1] / values.iloc[0] - 1) * 100
    years = (values.index[-1] - values.index[0]).days / 365.25
    cagr = ((values.iloc[-1] / values.iloc[0]) ** (1/years) - 1) * 100
    volatility = returns.std() * np.sqrt(252) * 100
    sharpe = (returns.mean() * 252) / (returns.std() * np.sqrt(252))
    
    # Maximum Drawdown
    cummax = values.cummax()
    drawdown = (values - cummax) / cummax
    max_dd = drawdown.min() * 100
    
    # Calmar Ratio
    calmar = cagr / abs(max_dd)
    
    return {
        'Total Return (%)': total_return,
        'CAGR (%)': cagr,
        'Volatility (%)': volatility,
        'Sharpe Ratio': sharpe,
        'Max Drawdown (%)': max_dd,
        'Calmar Ratio': calmar
    }

# Calculate metrics for all strategies
metrics_dict = {}
for col in portfolio_values.columns:
    metrics_dict[col] = calculate_metrics(portfolio_values[col])

metrics_df = pd.DataFrame(metrics_dict).T
print("\nPERFORMANCE COMPARISON")
print("="*80)
print(metrics_df.round(2))

In [ ]:
# Visualize backtest results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Portfolio values
ax1 = axes[0, 0]
for col in portfolio_values.columns:
    ax1.plot(portfolio_values.index, portfolio_values[col], label=col, linewidth=2)
ax1.set_xlabel('Date')
ax1.set_ylabel('Portfolio Value')
ax1.set_title('Portfolio Growth Comparison')
ax1.legend()
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# Drawdowns
ax2 = axes[0, 1]
for col in portfolio_values.columns:
    cummax = portfolio_values[col].cummax()
    drawdown = (portfolio_values[col] - cummax) / cummax * 100
    ax2.fill_between(drawdown.index, drawdown.values, 0, alpha=0.3, label=col)
ax2.set_xlabel('Date')
ax2.set_ylabel('Drawdown (%)')
ax2.set_title('Drawdown Comparison')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Risk-Return scatter
ax3 = axes[1, 0]
for col in portfolio_values.columns:
    ax3.scatter(metrics_df.loc[col, 'Volatility (%)'], 
                metrics_df.loc[col, 'CAGR (%)'],
                s=200, label=col)
    ax3.annotate(col, (metrics_df.loc[col, 'Volatility (%)'], 
                       metrics_df.loc[col, 'CAGR (%)']),
                xytext=(5, 5), textcoords='offset points')
ax3.set_xlabel('Volatility (%)')
ax3.set_ylabel('CAGR (%)')
ax3.set_title('Risk-Return Profile')
ax3.grid(True, alpha=0.3)

# Metrics comparison bar chart
ax4 = axes[1, 1]
metrics_to_plot = ['Sharpe Ratio', 'Calmar Ratio']
x = np.arange(len(portfolio_values.columns))
width = 0.35
ax4.bar(x - width/2, metrics_df['Sharpe Ratio'], width, label='Sharpe Ratio', alpha=0.8)
ax4.bar(x + width/2, metrics_df['Calmar Ratio'], width, label='Calmar Ratio', alpha=0.8)
ax4.set_xticks(x)
ax4.set_xticklabels(portfolio_values.columns, rotation=45)
ax4.set_ylabel('Ratio')
ax4.set_title('Risk-Adjusted Performance')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## Part 5: Extensions and Variations

### 5.1 Risk Budgeting (Custom Risk Allocations)

Instead of equal risk, allocate different risk budgets to each asset:
$$RC_i = b_i \quad \text{where} \quad \sum_i b_i = 1$$

In [ ]:
# Custom risk budgeting example
# Allocate 40% risk to equities, 30% to bonds, 10% to gold, 10% to REITs, 10% to international
custom_budgets = np.array([0.40, 0.30, 0.10, 0.10, 0.10])

print("CUSTOM RISK BUDGETING")
print("="*50)
print(f"Target Risk Budgets: {dict(zip(tickers, custom_budgets))}")

rb_weights = advanced_rp.risk_budgeting(custom_budgets)
rb_rc = get_risk_contributions(rb_weights, cov_matrix)

rb_df = pd.DataFrame({
    'Target RC (%)': custom_budgets * 100,
    'Actual RC (%)': rb_rc,
    'Weight (%)': rb_weights * 100
}, index=tickers)

print("\n", rb_df.round(2))

In [ ]:
# Visualize custom risk budgeting
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Target vs Actual Risk Contributions
ax1 = axes[0]
x = np.arange(len(tickers))
width = 0.35
ax1.bar(x - width/2, custom_budgets * 100, width, label='Target', alpha=0.8)
ax1.bar(x + width/2, rb_rc, width, label='Actual', alpha=0.8)
ax1.set_xticks(x)
ax1.set_xticklabels(tickers)
ax1.set_ylabel('Risk Contribution (%)')
ax1.set_title('Target vs Actual Risk Contributions')
ax1.legend()

# Weights
ax2 = axes[1]
ax2.bar(tickers, rb_weights * 100, color=plt.cm.Set2(np.linspace(0, 1, len(tickers))))
ax2.set_ylabel('Weight (%)')
ax2.set_title('Risk Budgeting Portfolio Weights')

plt.tight_layout()
plt.show()

### 5.2 Leveraged Risk Parity

Risk parity portfolios often have lower volatility than traditional portfolios.
To achieve target volatility, leverage can be applied:

$$w^{leveraged} = \frac{\sigma_{target}}{\sigma_{RP}} \times w^{RP}$$

In [ ]:
def leverage_portfolio(weights: np.ndarray, cov: np.ndarray, 
                       target_vol: float) -> tuple:
    """
    Apply leverage to achieve target volatility.
    
    Parameters:
    -----------
    weights : np.ndarray
        Portfolio weights
    cov : np.ndarray
        Covariance matrix
    target_vol : float
        Target annualized volatility
    
    Returns:
    --------
    tuple : (leveraged_weights, leverage_ratio)
    """
    current_vol = np.sqrt(weights @ cov @ weights)
    leverage = target_vol / current_vol
    leveraged_weights = weights * leverage
    
    return leveraged_weights, leverage

# Calculate leverage needed for different target volatilities
rp_vol = np.sqrt(erc_result['weights'] @ cov_matrix @ erc_result['weights'])
ew_vol = np.sqrt(equal_weights @ cov_matrix @ equal_weights)

print("LEVERAGED RISK PARITY")
print("="*50)
print(f"Unleveraged Risk Parity Volatility: {rp_vol*100:.2f}%")
print(f"Equal Weight Volatility: {ew_vol*100:.2f}%")

# Target equal weight volatility
lev_weights, leverage = leverage_portfolio(erc_result['weights'], cov_matrix, ew_vol)

print(f"\nLeverage needed to match EW volatility: {leverage:.2f}x")
print(f"\nLeveraged Risk Parity Weights:")
print(pd.DataFrame({
    'Asset': tickers,
    'Unleveraged (%)': erc_result['weights'] * 100,
    'Leveraged (%)': lev_weights * 100
}).set_index('Asset').round(2))

print(f"\nTotal portfolio exposure: {lev_weights.sum()*100:.1f}%")

### 5.3 Hierarchical Risk Parity (HRP)

An alternative approach that doesn't require matrix inversion and is more robust to estimation errors.

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from scipy.spatial.distance import squareform

class HierarchicalRiskParity:
    """
    Hierarchical Risk Parity (Lopez de Prado, 2016)
    """
    
    def __init__(self, returns: pd.DataFrame):
        self.returns = returns
        self.cov = returns.cov().values
        self.corr = returns.corr().values
        self.n_assets = len(returns.columns)
        self.asset_names = returns.columns.tolist()
    
    def _get_quasi_diag(self, link: np.ndarray) -> list:
        """Sort clustered items by distance."""
        link = link.astype(int)
        sorted_items = leaves_list(link)
        return sorted_items.tolist()
    
    def _get_cluster_var(self, cov: np.ndarray, cluster_items: list) -> float:
        """Calculate variance of cluster."""
        cov_slice = cov[np.ix_(cluster_items, cluster_items)]
        w = self._get_ivp(cov_slice)
        return np.dot(w, np.dot(cov_slice, w))
    
    def _get_ivp(self, cov: np.ndarray) -> np.ndarray:
        """Get inverse variance portfolio weights."""
        ivp = 1 / np.diag(cov)
        return ivp / ivp.sum()
    
    def _recursive_bisection(self, cov: np.ndarray, sorted_items: list) -> np.ndarray:
        """Compute HRP weights through recursive bisection."""
        w = np.ones(len(sorted_items))
        cluster_items = [sorted_items]
        
        while len(cluster_items) > 0:
            # Bisect
            cluster_items = [
                item[j:k] for item in cluster_items
                for j, k in ((0, len(item)//2), (len(item)//2, len(item)))
                if len(item) > 1
            ]
            
            # Allocate weights
            for i in range(0, len(cluster_items), 2):
                if i + 1 >= len(cluster_items):
                    break
                    
                cluster_0 = cluster_items[i]
                cluster_1 = cluster_items[i + 1]
                
                var_0 = self._get_cluster_var(cov, cluster_0)
                var_1 = self._get_cluster_var(cov, cluster_1)
                
                alpha = 1 - var_0 / (var_0 + var_1)
                
                w[cluster_0] *= alpha
                w[cluster_1] *= 1 - alpha
        
        return w
    
    def optimize(self) -> dict:
        """
        Compute HRP weights.
        
        Returns:
        --------
        dict : weights and clustering info
        """
        # Step 1: Compute distance matrix
        dist = np.sqrt((1 - self.corr) / 2)
        
        # Step 2: Hierarchical clustering
        dist_condensed = squareform(dist)
        link = linkage(dist_condensed, method='single')
        
        # Step 3: Quasi-diagonalization
        sorted_items = self._get_quasi_diag(link)
        
        # Step 4: Recursive bisection
        weights = self._recursive_bisection(self.cov, sorted_items)
        
        # Reorder weights to original order
        final_weights = np.zeros(self.n_assets)
        for i, item in enumerate(sorted_items):
            final_weights[item] = weights[i]
        
        return {
            'weights': final_weights,
            'linkage': link,
            'sorted_items': sorted_items
        }

In [ ]:
# Calculate HRP weights
hrp = HierarchicalRiskParity(returns)
hrp_result = hrp.optimize()

print("HIERARCHICAL RISK PARITY")
print("="*50)

# Compare all methods
all_weights = pd.DataFrame({
    'Equal Weight': equal_weights * 100,
    'Risk Parity': erc_result['weights'] * 100,
    'HRP': hrp_result['weights'] * 100,
    'Min Variance': backtester.minimum_variance(cov_matrix) * 100
}, index=tickers)

print(all_weights.round(2))

In [ ]:
# Visualize HRP clustering
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Dendrogram
ax1 = axes[0]
dendrogram(hrp_result['linkage'], labels=tickers, ax=ax1)
ax1.set_title('Hierarchical Clustering Dendrogram')
ax1.set_ylabel('Distance')

# Weight comparison
ax2 = axes[1]
all_weights.plot(kind='bar', ax=ax2, width=0.8)
ax2.set_ylabel('Weight (%)')
ax2.set_title('Portfolio Weight Comparison')
ax2.legend(loc='upper right')
ax2.set_xticklabels(tickers, rotation=0)

plt.tight_layout()
plt.show()

---
## Practice Exercises

### Exercise 1: Sector Risk Parity
Create a risk parity portfolio across different sectors (Technology, Healthcare, Financials, Energy, Consumer).

In [ ]:
# Exercise 1: Your code here
sector_tickers = ['XLK', 'XLV', 'XLF', 'XLE', 'XLY']  # Sector ETFs

# Download data and implement risk parity
# ...

print("TODO: Implement sector risk parity")

### Exercise 2: Dynamic Risk Parity with Regime Detection
Implement a risk parity strategy that adjusts risk budgets based on market regime (bull/bear).

In [ ]:
# Exercise 2: Your code here
def detect_regime(returns: pd.Series, lookback: int = 60) -> str:
    """
    Detect market regime based on recent performance.
    
    Returns 'bull' or 'bear'
    """
    # TODO: Implement regime detection
    pass

print("TODO: Implement regime-aware risk parity")

### Exercise 3: Transaction Cost Analysis
Analyze the impact of transaction costs on risk parity rebalancing.

In [ ]:
# Exercise 3: Your code here
def calculate_turnover(weights_history: list) -> float:
    """
    Calculate average portfolio turnover.
    """
    # TODO: Implement turnover calculation
    pass

print("TODO: Analyze transaction costs")

---
## Interview Questions

### Conceptual
1. **What is the key insight behind risk parity?**
   - Equal capital allocation ≠ equal risk allocation
   - Low-risk assets need more weight to contribute equally to risk

2. **Why does risk parity often overweight bonds?**
   - Bonds typically have lower volatility
   - Need larger allocation to achieve equal risk contribution

3. **What are the criticisms of risk parity?**
   - Ignores expected returns (focuses only on risk)
   - May require leverage to achieve competitive returns
   - Sensitive to correlation assumptions
   - Can have concentrated positions in low-vol assets

### Technical
4. **Derive the risk contribution formula.**
   - Start from portfolio variance, take partial derivative, apply Euler's theorem

5. **How do you handle negative weights in risk parity?**
   - Add long-only constraints
   - Use bounded optimization

6. **Compare Risk Parity vs. Minimum Variance.**
   - MinVar minimizes total risk; RP equalizes risk contributions
   - MinVar can concentrate in low-vol assets; RP diversifies risk

### Practical
7. **How would you implement risk parity for 500 assets?**
   - Use efficient optimization methods (CCD, Newton)
   - Consider HRP for large-scale portfolios
   - Factor-based risk parity as alternative

8. **How do you handle estimation error in the covariance matrix?**
   - Shrinkage estimators
   - Factor models
   - Robust optimization

---
## Summary

### Key Concepts
1. **Risk Contribution**: How much each asset contributes to portfolio risk
2. **Equal Risk Contribution (ERC)**: Each asset contributes equally to risk
3. **Risk Budgeting**: Custom risk allocations to each asset
4. **Hierarchical Risk Parity**: Cluster-based approach, more robust

### Implementation Methods
- SLSQP optimization
- Newton-Raphson
- Cyclical Coordinate Descent
- Inverse Volatility (approximation)

### Key Formulas
- Marginal Risk: $MRC_i = \frac{(\Sigma w)_i}{\sigma_p}$
- Risk Contribution: $RC_i = \frac{w_i (\Sigma w)_i}{w^T \Sigma w}$
- ERC Condition: $RC_i = 1/n \; \forall i$

### Next Steps
- Day 4: Black-Litterman Model
- Explore factor-based risk parity
- Study tail-risk parity